# Data Analysis: Video Game Sales - Python / SQL Project

Welcome to my Video Game Sales Data Analysis Project! It is currently Sunday and I don't have much to do. So to rid myself of boredom, I decided to analyze random data. My friends and I are gamers, so I decided to analyze data on video games. In this project, I'll answer questions like "which consoles is the most successful", "which console is the most active", "which console is most successful in specific regions", and many more. Enjoy!

In [65]:
import mysql.connector
import pandas as pd

### To start the project, I would first need to connect to the SQL database the houses the data I need

For potential employers --> Most of my SQL expertise is built on MySQL and MySQL Workbench. I started off using SQLite, but found MySQL / Workbench more comfortable. 

In [66]:
mydb = mysql.connector.connect(host="localhost",
                               user="root",
                               password="NonDucorDuco321",
                               database="vgsales"
)

print(mydb)

In [67]:
mycursor = mydb.cursor()

### Before starting the analysis, I want to display the dataframe for easy reference

In [68]:
mycursor.execute("SELECT * FROM vgsales")
myresult = mycursor.fetchall()
dataframe = pd.DataFrame(myresult)
dataframe.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


### Obviously the dataframe above is missing a key feature: The headers. Luckily the dataset (downloaded from Kaggle.com) supplies me with the headers and I can easily input them into the dataframe

In [69]:
dataframe.columns = ['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales',
                    'JP_Sales', 'Other_Sales', 'Global Sales']
dataframe.head(10)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31
